In [1]:
import numpy as npy, os, matplotlib.pyplot as plt

In [2]:
cd ../../ImageSets/

/home/tanmay/Research/Code/HierarchicalTaskRL/ImageSets


In [3]:
images = npy.load("NEWSTRIPS.npy")
images[images==2]=1.
images[images==-1]=0.

labels = npy.load("NEWSTRIPS.npy")

In [4]:
def infogain(y,a,l):
#     if a==1:
#         s1=y[:l,:]
#         s2=y[l:,:]        
#     if a==0:
#         s1=y[:,:l]
#         s2=y[:,l:]
    if a==0:
        s1=y[:l,:]
        s2=y[l:,:]        
    if a==1:
        s1=y[:,:l]
        s2=y[:,l:]

    ysize = y.shape[0]*y.shape[1]
    ps1 = float( s1.shape[0]*s1.shape[1])/ysize
    ps2 = float(s2.shape[0]*s2.shape[1])/ysize
    return entropy(y)-(ps1*entropy(s1)+ps2*entropy(s2))

In [5]:
def entropy(y):     
    nones = npy.count_nonzero(y)
    ysize = y.shape[0]*y.shape[1]    
    nzeros = ysize-nones
    pz = float(nzeros)/ysize
    po = float(nones)/ysize
    if pz==0 or po==0: 
        return 0.                
    return -(pz*npy.log2(pz)+po*npy.log2(po))  

In [6]:
def bestsplit(x):
    maxval = -1           
    chosen_a = -1                             
    chosen_l = -1                                       
    print("Shape of image:",x.shape)
    for a_val in range(2):         
        if a_val==0:
            limval = x.shape[0]-1
        if a_val==1:
            limval = x.shape[1]-1
        for l_val in range(1,limval):
            
            ig = infogain(x,a_val,l_val)   
#             print(a_val,l_val,ig,limval, chosen_a,chosen_l, maxval)
            if ig>maxval:            
                maxval=ig                          
                chosen_a = a_val                  
                chosen_l = l_val   
    if maxval==0:
        print("No entropy reducing splits.")
        return 1
    return chosen_a,chosen_l

In [7]:
def return_splits(x,a,l):
    if a==0:
        s1=x[:l,:]
        s2=x[l:,:]
    if a==1:
        s1=x[:,:l]
        s2=x[:,l:]
    return s1,s2

In [33]:
r_values = npy.zeros(500)

# For all images:
for i in range(500):
    a,l = bestsplit(images[i])
    s1,s2 = return_splits(labels[i],a,l)
    
    s1_sum = s1.sum()
    if s1_sum==0:
        s1_val = 0
    else:
        s1_val = (s1_sum/abs(s1_sum))*s1_sum
    
    s2_sum = s2.sum()
    if s2_sum==0:
        s2_val = 0
    else:
        s2_val = (s2_sum/abs(s2_sum))*s2_sum          
    
    r_values[i] = (s1_val+s2_val)/(256**2)
    
    print(i)

Shape of image: (256, 256)
0
Shape of image: (256, 256)
1
Shape of image: (256, 256)
2
Shape of image: (256, 256)
3
Shape of image: (256, 256)
4
Shape of image: (256, 256)
5
Shape of image: (256, 256)
6
Shape of image: (256, 256)
7
Shape of image: (256, 256)
8
Shape of image: (256, 256)
9
Shape of image: (256, 256)
10
Shape of image: (256, 256)
11
Shape of image: (256, 256)
12
Shape of image: (256, 256)
13
Shape of image: (256, 256)
14
Shape of image: (256, 256)
15
Shape of image: (256, 256)
16
Shape of image: (256, 256)
17
Shape of image: (256, 256)
18
Shape of image: (256, 256)
19
Shape of image: (256, 256)
20
Shape of image: (256, 256)
21
Shape of image: (256, 256)
22
Shape of image: (256, 256)
23
Shape of image: (256, 256)
24
Shape of image: (256, 256)
25
Shape of image: (256, 256)
26
Shape of image: (256, 256)
27
Shape of image: (256, 256)
28
Shape of image: (256, 256)
29
Shape of image: (256, 256)
30
Shape of image: (256, 256)
31
Shape of image: (256, 256)
32
Shape of image: (256

267
Shape of image: (256, 256)
268
Shape of image: (256, 256)
269
Shape of image: (256, 256)
270
Shape of image: (256, 256)
271
Shape of image: (256, 256)
272
Shape of image: (256, 256)
273
Shape of image: (256, 256)
274
Shape of image: (256, 256)
275
Shape of image: (256, 256)
276
Shape of image: (256, 256)
277
Shape of image: (256, 256)
278
Shape of image: (256, 256)
279
Shape of image: (256, 256)
280
Shape of image: (256, 256)
281
Shape of image: (256, 256)
282
Shape of image: (256, 256)
283
Shape of image: (256, 256)
284
Shape of image: (256, 256)
285
Shape of image: (256, 256)
286
Shape of image: (256, 256)
287
Shape of image: (256, 256)
288
Shape of image: (256, 256)
289
Shape of image: (256, 256)
290
Shape of image: (256, 256)
291
Shape of image: (256, 256)
292
Shape of image: (256, 256)
293
Shape of image: (256, 256)
294
Shape of image: (256, 256)
295
Shape of image: (256, 256)
296
Shape of image: (256, 256)
297
Shape of image: (256, 256)
298
Shape of image: (256, 256)
299
Shap

In [39]:
from sklearn.metrics import jaccard_similarity_score
j_values = npy.zeros(500)

# For all images:
for i in range(500):
    a,l = bestsplit(images[i])
    s1,s2 = return_splits(labels[i],a,l)
    preds = npy.zeros((256,256))
    s1_mean = s1.mean()
    if s1_mean==0:
        preds[:l,:] = 1.
    else:
        preds[:l,:] = s1_mean/abs(s1_mean)
    s2_mean = s2.mean()
    if s2_mean==0:
        preds[l:,:] = 1.
    else:
        preds[l:,:] = s2_mean/abs(s2_mean)
        
    j_values[i] = jaccard_similarity_score(labels[i].reshape((256**2)),preds.reshape((256**2)))
    print(i)

Shape of image: (256, 256)
0
Shape of image: (256, 256)
1
Shape of image: (256, 256)
2
Shape of image: (256, 256)
3
Shape of image: (256, 256)
4
Shape of image: (256, 256)
5
Shape of image: (256, 256)
6
Shape of image: (256, 256)
7
Shape of image: (256, 256)
8
Shape of image: (256, 256)
9
Shape of image: (256, 256)
10
Shape of image: (256, 256)
11
Shape of image: (256, 256)
12
Shape of image: (256, 256)
13
Shape of image: (256, 256)
14
Shape of image: (256, 256)
15
Shape of image: (256, 256)
16
Shape of image: (256, 256)
17
Shape of image: (256, 256)
18
Shape of image: (256, 256)
19
Shape of image: (256, 256)
20
Shape of image: (256, 256)
21
Shape of image: (256, 256)
22
Shape of image: (256, 256)
23
Shape of image: (256, 256)
24
Shape of image: (256, 256)
25
Shape of image: (256, 256)
26
Shape of image: (256, 256)
27
Shape of image: (256, 256)
28
Shape of image: (256, 256)
29
Shape of image: (256, 256)
30
Shape of image: (256, 256)
31
Shape of image: (256, 256)
32
Shape of image: (256

268
Shape of image: (256, 256)
269
Shape of image: (256, 256)
270
Shape of image: (256, 256)
271
Shape of image: (256, 256)
272
Shape of image: (256, 256)
273
Shape of image: (256, 256)
274
Shape of image: (256, 256)
275
Shape of image: (256, 256)
276
Shape of image: (256, 256)
277
Shape of image: (256, 256)
278
Shape of image: (256, 256)
279
Shape of image: (256, 256)
280
Shape of image: (256, 256)
281
Shape of image: (256, 256)
282
Shape of image: (256, 256)
283
Shape of image: (256, 256)
284
Shape of image: (256, 256)
285
Shape of image: (256, 256)
286
Shape of image: (256, 256)
287
Shape of image: (256, 256)
288
Shape of image: (256, 256)
289
Shape of image: (256, 256)
290
Shape of image: (256, 256)
291
Shape of image: (256, 256)
292
Shape of image: (256, 256)
293
Shape of image: (256, 256)
294
Shape of image: (256, 256)
295
Shape of image: (256, 256)
296
Shape of image: (256, 256)
297
Shape of image: (256, 256)
298
Shape of image: (256, 256)
299
Shape of image: (256, 256)
300
Shap

In [8]:
# Under a random split location policy, with optimal assignments.


random_r_values = npy.zeros((500,255))

# For all images:
for i in range(500):
    
    for l in range(255):
        
        s1, s2 = return_splits(labels[i],0,l)
    
        s1_sum = s1.sum()
        if s1_sum==0:
            s1_val = 0
        else:
            s1_val = (s1_sum/abs(s1_sum))*s1_sum

        s2_sum = s2.sum()
        if s2_sum==0:
            s2_val = 0
        else:
            s2_val = (s2_sum/abs(s2_sum))*s2_sum          
    
        random_r_values[i,l] = (s1_val+s2_val)/(256**2)
    
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
random_r_values.mean()

0.5365818014705882

In [40]:
j_values

array([0.89453125, 0.8515625 , 0.86328125, 0.890625  , 0.8671875 ,
       0.82421875, 0.8984375 , 0.8671875 , 0.8046875 , 0.82421875,
       0.8984375 , 0.8984375 , 0.8828125 , 0.69921875, 0.7578125 ,
       0.796875  , 0.87890625, 0.8125    , 0.8671875 , 0.86328125,
       0.8984375 , 0.8984375 , 0.78515625, 0.75390625, 0.8046875 ,
       0.828125  , 0.7890625 , 0.8515625 , 0.8515625 , 0.796875  ,
       0.82421875, 0.8828125 , 0.83984375, 0.8515625 , 0.87890625,
       0.84375   , 0.78515625, 0.89453125, 0.8828125 , 0.86328125,
       0.80078125, 0.89453125, 0.87109375, 0.84375   , 0.89453125,
       0.8828125 , 0.6953125 , 0.8125    , 0.8046875 , 0.89453125,
       0.890625  , 0.6875    , 0.765625  , 0.87109375, 0.8203125 ,
       0.875     , 0.8203125 , 0.85546875, 0.85546875, 0.87109375,
       0.87890625, 0.85546875, 0.8515625 , 0.87109375, 0.84765625,
       0.83984375, 0.78515625, 0.80859375, 0.89453125, 0.89453125,
       0.8515625 , 0.89453125, 0.859375  , 0.890625  , 0.88281

In [41]:
j_values.mean()

0.83725

In [35]:
r_values.mean()

0.6745

In [38]:
tanvalues = npy.zeros((500))

for i in range(500):
    tanvalues[i] = npy.math.tan(r_values[i])
tanvalues,tanvalues.mean()

(array([1.00735566, 0.84764453, 0.88874597, 0.9917379 , 0.90282739,
        0.75774184, 1.0232212 , 0.90282739, 0.69798896, 0.75774184,
        1.0232212 , 1.0232212 , 0.96121551, 0.42095263, 0.56676707,
        0.67500049, 0.9462965 , 0.72148444, 0.90282739, 0.88874597,
        1.0232212 , 1.0232212 , 0.64140953, 0.55649029, 0.69798896,
        0.77011355, 0.6524919 , 0.84764453, 0.84764453, 0.67500049,
        0.75774184, 0.96121551, 0.80814465, 0.84764453, 0.9462965 ,
        0.8211418 , 0.64140953, 1.00735566, 0.96121551, 0.88874597,
        0.68643308, 1.00735566, 0.91710886, 0.8211418 , 1.00735566,
        0.96121551, 0.41178571, 0.72148444, 0.69798896, 1.00735566,
        0.9917379 , 0.39362658, 0.58759737, 0.91710886, 0.74551574,
        0.93159646, 0.74551574, 0.8611601 , 0.8611601 , 0.91710886,
        0.9462965 , 0.8611601 , 0.84764453, 0.91710886, 0.83430679,
        0.80814465, 0.64140953, 0.70967157, 1.00735566, 1.00735566,
        0.84764453, 1.00735566, 0.87485876, 0.99

In [ ]:
a,l = bestsplit(s2)
s1,s2 = return_splits(s2,a,l)

In [ ]:
plt.imshow(s1)
plt.show()

In [ ]:
a,l = bestsplit(s1)
s1,s2 = return_splits(s1,a,l)

In [ ]:
plt.imshow(s2)
plt.show()

In [ ]:
a,l = bestsplit(s2)
s1,s2 = return_splits(s2,a,l)

In [ ]:
plt.imshow(s1)
plt.show()

In [ ]:
a,l = bestsplit(s1)
s3,s4 = return_splits(s1,a,l)


In [ ]:
plt.imshow(s3)
plt.show()

In [ ]:
plt.imshow(s4)
plt.show()

In [ ]:
a,l = bestsplit(s3)
a,l